# DAA M07 Natural language processing
## Case study 7.1 - Classifying automative reviews

### Purpose:
This notebook walks through the use of NLP to analyse customer sentiment. It forms the basis of Case Study 1 in Module 7 and is referenced in multiple exercises throughout the module.

The objective of the case study is to identify 'complaints' from a sample of YELP reviews for automotive companies. YELP is a North American review service where customers review different service providers. Reviewers write a text review and are then asked to rate the service on a five point scale.

### References:
The dataset that is used in this case study was sourced from a Kaggle competition in which YELP provided a set of reviews: https://www.kaggle.com/yelp-dataset/yelp-dataset.

It contains information about businesses across 8 metropolitan areas in the USA and Canada.

For this case study, the dataset has been refined to contain 50,000 reviews for companies tagged as 'automotive'.

## Packages
This section installs packages that will be required for this exercise/case study.

In [2]:
import json # Json is used to load json files.
import os # os is used to join paths for windows and unix use.
import random # For random number generation.
import time # Useful for timing processes.

from collections import Counter # Counter provides a quick way to add items in a list.
from collections import defaultdict # Enhanced mapping dictionary that can take a default value.
import numpy as np # For mathematical operations.
import matplotlib.pyplot as plt # For plotting.

from nltk.tokenize import word_tokenize
  # 'word_tokenize' is a tokeniser for natural language.
  # There are many such options available.
import nltk
# nltk.download('punkt')

# Sklearn has many useful machine learning packages.
# The following are used in the notebook.
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

# Used to upload a file to Colab.
from google.colab import files

# Package used in the confusion matrix function.
import itertools


/home/analytics-user/anaconda3/lib/python3.6/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


## Functions
This section defines functions that will be used for this exercise/case study.

In [3]:
# Define a function to load a json data file.
def load_json(data_set_name, dir = '', verbose=True):
    '''
    This is a function to load the yelp dataset which is stored in json format.
    Use the optional dir argument if the file is not in the same directory
    as the notebook.
    '''
    if verbose:
        print('loading', data_set_name, 'data ', end = '')
        # If the user has set Verbose = True (or omitted this argument),
        # these lines of code provide a message to the person running the code
        # that the dataset is being loaded.
    
    filename = os.path.join(dir, data_set_name)
    f = open(filename, encoding='utf8')

    data_set = list()
    for counter, line in enumerate(f):
        data_set.append(json.loads(line))
        
        if verbose and ((counter % 100000) == 0):
            print('.', end='')
            # If the dataset is very big, these lines of code print '.'s to
            # show that the import step is progressing.

    f.close()
    
    if verbose: 
        print(' {:,} observations have been imported.'.format(len(data_set)))
        # This statement summarises the number of observations that have been
        # imported from the dataset.
    
    return data_set

In [4]:
# Define a function to print the Yelp reviews neatly.
def print_review(review):
    for item in ['business_id', 'stars', 'funny', 'review_id', 'date', 'useful',
                 'user_id', 'cool']:
        print('{:<12s}: {:}'.format(item, review[item]))
    print ('text: "', review['text'], '"', sep = '')
    # The default for print() is to put a space between each argument. 
    # Here sep='' is used to overrule this.
    print()
    
    return

In [5]:
# Define a function to select reviews with a certain rating (number of stars).
def sample_with_rating(review_data, rating, sample_size):
    select_data = [data_row for data_row in review_data \
                   if data_row['stars'] == rating]
    review_sample = random.sample(select_data, sample_size)
    return review_sample

In [6]:
# Define a function to format the summary of review ratings.
def print_review_count(review_count):
    head_format = '{:<7s}  {:>10s} {:>10s}'
    row_format = '{:<7}  {:>10,} {:>10.1%}'
    
    print(head_format.format('Stars', 'Total', 'Percent'))
    print(head_format.format('-----', '-----', '-------'))
        
    total_reviews = sum([count for count in review_count.values()])

    # Add the percent to a dictionary indexed in the same way.
    review_perc = dict()
    for rating, review in review_count.items():
        review_perc[rating] = review/total_reviews
    
    for k in range(5, 0, -1):
        print(row_format.format(('*'*k), review_count[k], review_perc[k]))

    print(head_format.format('','-------','-------'))
    print(row_format.format('Total', total_reviews, 1))

In [7]:
# Define a function to print some example reviews based on a user input
# of 'min', 'max', 'average' etc summary function.
def print_certain_reviews(data, summ_func, descr):
    # The 'summ_func' will be 'min', 'max', 'average', 'median' etc.
    certain_review = summ_func([length for (review, length) in data])
    reviews = [review for (review, length) in data if length == certain_review]
    review_count = len(reviews)
    if review_count == 1:
        print("Below is the 1 {:s} review of {:,} words:".format(descr, certain_review))
    else:
        print("Below are the {:,} {:s} reviews of {:,} words:".format(review_count, descr, certain_review))
      
    for review in reviews:
        print('"',review,'"')
    print()

In [8]:
# Define a function to perform the Bag of Words vectorisation.
def vectorise_data(data_set, vocab):
    matrix = []
    for k, review in enumerate(data_set):
        if k % 1000 == 0:
            print('.', end='')
            # Prints '.' every 1,000 reviews so that you
            # know the code is still running.
        token_count = Counter(review['tokens'])
            # Create a count of all the unique tokens in the review.
        matrix.append([token_count.get(token, 0) for token in vocab])
            # The token counts for the review are added to 'matrix'. 
            # The 'get' function of Counter is used to return a 0
            # for any tokens in the vocabulary that are not in a review.
    print()
    
    return matrix

In [9]:
# Define a function to calculate a range of success metrics for a fitted model.
def return_metrics(y, y_pred):
    scores = dict()
    scores['Accuracy'] = metrics.accuracy_score(y, y_pred)
    scores['F1-score'] = metrics.f1_score(y, y_pred, average='binary', pos_label=True)
    scores['Precision'] = metrics.precision_score(y, y_pred, pos_label=True)
    scores['Recall'] = metrics.recall_score(y, y_pred, pos_label=True)   
    scores['Confusion Matrix'] = metrics.confusion_matrix(y, y_pred)

    return scores

In [10]:
# Define a function to plot confusion matrices in an easy to visualise format.
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    '''
    This function prints and plots a nicely formatted confusion matrix.
    '''
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment='center',
                 color='white' if cm[i, j] > thresh else 'black')

    plt.tight_layout()
    plt.ylabel('True response')
    plt.xlabel('Predicted response')

In [11]:
# Define a function to print out the success metrics of a model.
def print_scores(scores, confused = False):
    print(('{:^9} '*4).format(*scores.keys()))
    print(('{:^9.1%} '*4).format(*scores.values()))

    if confused:
            plot_confusion_matrix(scores['Confusion Matrix'],
                                   classes = ['Non-complaint', 'Complaint'])

## Data
This section:
- imports the data that will be used in the modelling;
- explores the data; and
- prepares the data for modelling.

### Import data


In [ ]:
# Mount your google drive to import the required data
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)

In [12]:
# Specify the Google Drive folder that your datasets are saved in.
# infolder = '/content/gdrive/My Drive/DAA datasets/'
infolder = '/data/projects/personal/daa_2021s1/07 Natural Language Processing'

# Specify the name of the dataset to be imported.
file = 'DAA_M07_CS1_data.json'

# Upload the required YELP file from your Google Drive folder.
review_data = load_json(file,dir=infolder)

# Alternatively, use the code below to upload the file
# from your computer. This method is slower than the Google Drive method.

#files.upload()
  # When prompted, select the DAA_M07_CS1_data.json dataset to upload.
  # Note that this may take some time to upload as the file is large.
#review_data = load_json('DAA_M07_CS1_data.json')

loading DAA_M07_CS1_data.json data . 50,000 observations have been imported.


In [15]:
review_data

[{'business_id': '_iGvLfEsqDwPUxRUAe6tUw',
  'cool': 0,
  'date': '2017-06-28 00:39:18',
  'funny': 0,
  'review_id': 'Hy-gUXQh3RVhE8FLH9Dt4Q',
  'stars': 1.0,
  'text': "Tried to have my car repaired.  Even made an appointment.  I was told that all my appointment was for is to meet the service advisor, not get any work completed on my car.  Such a waste of time.  Even talked to the Service Manager, he didn't care.  Since that was the attitude of the service department, I can only expect that from the mechanics that would have worked on my car.  Went to another dealership, made an appointment and my car went in to be repaired.  Not told to come back later and they might get to my car. Very poor service.",
  'useful': 0,
  'user_id': 'kMkWON2lmw0s-M-fwiuByg'},
 {'business_id': 'l4ByLNdIcsjgnglB1aXQQQ',
  'cool': 0,
  'date': '2018-04-15 14:54:30',
  'funny': 0,
  'review_id': 'BnB8z3d1BiLyqdAwgQnejw',
  'stars': 1.0,
  'text': "I can't tell you how angry I am right now.  I went to Hunte

### Explore data (EDA)
The following data exploration is performed below:
- look at the features available in the data;
- show some example reviews;
- see how many reviews there are in each rating category; and
- look at how long the reviews are.


In [14]:
# Extract the feature names from the data.
features = {key for review in review_data for key in review.keys()}
  # The '{}' brackets are used to define a set that contains unique entries.
print(features)

# The feature names could also be obtained used a for loop per the code below.
# This is a less efficient way of extracting the information.
features2 = set() # This creates an empty set.
for review in review_data:
    for key in review.keys():
        features2.add(key)
print(features2)

{'cool', 'stars', 'funny', 'text', 'business_id', 'user_id', 'review_id', 'date', 'useful'}
{'cool', 'stars', 'funny', 'text', 'business_id', 'user_id', 'review_id', 'date', 'useful'}


In [16]:
# Print a sample of 5 random reviews from the full dataset.
review_sample = random.sample(review_data, 5)
  # The 'sample' method in the 'random' package selects, without replacement,
  # the number of items you want from a list.

for review in review_sample:
    print_review(review)

business_id : 3jjZftEYMzTLzlp8GKSKnQ
stars       : 5.0
funny       : 0
review_id   : rpPj8IVmp-ACjeAZHNzbWw
date        : 2019-06-16 14:53:36
useful      : 0
user_id     : voRBCbP-3MYHu5fx1qlr4A
cool        : 0
text: "I had a chip in my windshield that looked like is was about to go crazy and crack in all directions. I called Safelite and they came out the next morning and the technician Bobby did a great job, very polite and I couldn't even really see where the chip was when he was done, thanks Bobby, if I have another issue with my windshield I'm hoping that he is my technician....."

business_id : i0SM6xkFajG8YQqjlr3DMQ
stars       : 1.0
funny       : 0
review_id   : 9U5c48zjEWzFb4shkGTL-Q
date        : 2017-04-18 14:35:22
useful      : 2
user_id     : Q3Lvq9HAnsXfAyHwtBCE-g
cool        : 0
text: "Negative stars if possible . Dropped my car off at 9 am and appointment I had  for a hitch installation should have taken 2 hours ... 10 hours later i get my car.  I had to call twice to e

In [17]:
# Print 5 random reviews with 1 star ratings using the
# 'sample_with_rating' function defined at the top of the notebook.
review_sample = sample_with_rating(review_data, rating=1, sample_size=5)
for review in review_sample:
    print_review(review)

business_id : wb2BcN0YLGwdyIoWRHD_FA
stars       : 1.0
funny       : 0
review_id   : btlxXd8BmEDYWvXFeBzcgg
date        : 2018-03-02 22:29:12
useful      : 0
user_id     : qQtWShadkAukIsY7O_FU1g
cool        : 0
text: "If you want it to take an hour for car to be washed - this is the place to go! My car sat in the sun air drying while there were four other cars in front of it and no one was working on the line. They were going two at a time - I spoke with the manager and he said that's just how the process goes and it can take up to 30 min for a car wash. Also the workers just go so slow and chat with one another as of here aren't 30 other people waiting."

business_id : wnG7pj-_-EMBqLCT-zqJXw
stars       : 1.0
funny       : 2
review_id   : WRffVscerfFwQborr66cWQ
date        : 2016-10-17 16:13:00
useful      : 0
user_id     : zxGB-9Me-VZIsrsdb_SJug
cool        : 0
text: "How can I give 0 stars?  I brought in my truck for an electrical problem and gave them 4 days, which I was told was m

In [18]:
# Count the number of reviews in each rating category.
# This is done using a counter as an efficient method to iterate through items.

review_count = Counter([review['stars'] for review in review_data])
  # A list constructor is used to produce a list of all the star ratings
  # and the Counter function is then used to count how many reviews there are
  # in each star rating group.
  
print_review_count(review_count)

Stars         Total    Percent
-----         -----    -------
*****        27,101      54.2%
****          3,915       7.8%
***           1,790       3.6%
**            2,470       4.9%
*            14,724      29.4%
            -------    -------
Total        50,000     100.0%


In [20]:
# Check the length of different reviews.
# A list constructor is used to produce a list of how long each review is
# in characters. 
review_length_characters = [len(review['text']) for review in review_data]

# Print summary statistics for the number of characters in each review.
print('The longest character length in a review is {:,}.'.format(max(review_length_characters)))
print('The shortest character length in a review is {:,}.'.format(min(review_length_characters)))
print('The average character length of reviews is {:.0f}.'.format(np.mean(review_length_characters)))
print('The median character length of reviews is {:.0f}.'.format(np.median(review_length_characters)))
print()

# A list constructor is used to produce a list of how long each review is
# in words.
review_length_words = [len(review['text'].split()) for review in review_data]
  # The str.split() function breaks a string by approximate word breaks. 

## Print summary statistics for the number of words in each review.
print('The longest word length in a review is {:,}.'.format(max(review_length_words)))
print('The shortest word length in a review is {:,}.'.format(min(review_length_words)))
print('The average word length of reviews is {:.0f}.'.format(np.mean(review_length_words)))
print('The median word lenth of reviews is {:.0f}.'.format(np.median(review_length_words)))

The longest character length in a review is 5,000.
The shortest character length in a review is 1.
The average character length of reviews is 630.
The median character length of reviews is 436.

The longest word length in a review is 1,051.
The shortest word length in a review is 1.
The average word length of reviews is 119.
The median word lenth of reviews is 82.


In [21]:
# Print some examples of the shortest and longest reviews.
review_and_length = [(review['text'],len(review['text'].split())) for review in review_data]

print_certain_reviews(review_and_length, min, 'shortest')
print_certain_reviews(review_and_length, max, 'longest')

Below are the 4 shortest reviews of 1 words:
" L "
" Great! "
" :) "
" :) "

Below is the 1 longest review of 1,051 words:
" CAUTION!!!!!! DO NOT GO HERE!!!! If I could I'd give this place a -5 star!!! That's the first red flag u should know. I have a long story & from reading ALOT of reviews id say about 98% of them r poor reviews about this place. About 5 of them r very similar to mine. So here's my horrible story and experience...It was last month in Oct of 2015 ,I've been hearing about the Tent sale on 83rd between Bell rd & T-bird. Sale started Oct 22nd. So I have a 2014 Crystler 200 & it was my first car I've owned, by this time I had it for about 7 months. My heart was set on a Kia Optima but ended up settling for the 200. So when we get there we are given a person to take us around to look at cars. We were with 2 guys, Brian & Tavo from Vans Chevy. Before I go on, Tavo is an amazing nice guy that is very honest, very helpful so I have nothing bad to say about Tavo. Anyways, Bri

In [22]:
# Print the 20 most common words across the whole corpus of complaints.
word_count = Counter([word for review in review_data for word in review['text'].split()])
print("{:<6} {:>12}".format("Word", "Count"))
print("{:<6} {:>12}".format("----", "----------"))
for word, count in word_count.most_common(20):
    # 'most_common' is a helpful method that can be applied to Counter.
    print("{:<6} {:>12,}".format(word, count))

Word          Count
----     ----------
the         227,409
and         201,289
I           194,839
to          183,683
a           138,253
was          92,536
my           89,669
for          70,787
in           65,321
of           65,069
that         55,917
they         55,752
it           54,209
car          49,644
me           48,401
with         46,183
is           45,623
on           44,690
have         39,543
you          34,432


### Prepare data
This section performs the following NLP tasks:
- tokenisation;
- splitting the data into training and test sets;
- defining the response variable;
- creating the vocabulary;
- cleaning; and
- vectorisation.


#### Tokenise

In the data exploration sections above, tokenising was performed using the Python str function 'split'. This function splits on white space and does not, for example, tokenise end of sentence markers such as '?' or '.' .

The 'word_tokenize' tokeniser from the nltk package can provide a better tokenisation outcome than the 'split' function because it identifies items such as end of sentence markers.

In [23]:
t0 = time.time()
  # This sets the starting time so the time taken
  # to tokenise can be measured.

# Tokenise the reviews using 'word_tokeniser'.
for counter, review in enumerate(review_data):
    if counter % 1000 == 1:
        print ('.', end='')
        # This prints a '.' every 1,000 reviews to reassure
        # you that the code is still running.
    review['tokens'] = word_tokenize(review['text'])
        # word_tokenize is an nltk tokeniser.
        # The set of tokens from each review is added to the review
        # dictionary under a new dictionary index 'tokens'.
print()

print("Tokenising took {:.0f} seconds".format(time.time()-t0))
print()

# Look at five sample sets of tokens across the whole data set.
review_sample = random.sample(review_data, 5)
for review in review_sample:
    print(review['text'])
    print(review['tokens'], '\n')


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  Searched in:
    - '/home/analytics-user/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/home/analytics-user/anaconda3/nltk_data'
    - '/home/analytics-user/anaconda3/share/nltk_data'
    - '/home/analytics-user/anaconda3/lib/nltk_data'
    - ''
**********************************************************************


#### Clean data
The 'word_tokenize' tokeniser distinguishes between upper and lower case words such that 'Great' and 'great' are considered as two separate tokens. For this data set, such distinction between case may not be warranted and the model might perform better if all words are converted to their lower case form.
This is done in the code below.

In [ ]:
# Convert the tokens for each review to lower case.
for review in review_data:
    review['tokens'] = [token.lower() for token in review['tokens']]

# Look at another five sample sets of lower case tokens across the whole data set.
review_sample = random.sample(review_data, 5)
for review in review_sample:
    print(review['text'])
    print(review['tokens'], '\n')

#### Split into training and test data

Now the data is partioned into a training and a test set. The chosen partion is 50% training and 50% test. This is because the dataset is quite large, with 
50,0000 reviews and having a large training set can cause Colab to crash when fitting the model. In practice, if RAM was not an issue, you might want to fit the model with more training data using, for example, an 80%/20% split.

Note that as has been shown in previous case studies in this subject, you might want to further split the training data into a training and validation set. The validation dataset can then be used to select different hyperparameters for your model. Alternatively, cross-validation can be used on the training data to help you select these hyperparameters.

This further splitting of the training data into a training and validation data set has been omitted from this case study.

In [ ]:
train, test = train_test_split(review_data, test_size = 0.5)
  # The Sklearn package 'train_test_split' is used to automate this step. 
print(('{:>10s}'*3).format('training', 'test', 'total'))
print(('{:10,}'*3).format(len(train), len(test), len(review_data)))

#### Define response

In [ ]:
# Define the response variable as 1 (a complaint) if the number of stars given
# is 1 or 2. Otherwise define the response as 0 (non complaint).
train_y = [review['stars']<3 for review in train]
test_y = [review['stars']<3 for review in test]

# Check the total number of complaints is still the same
# across the training and test sets.
print(sum(train_y+test_y))
print(review_count[1]+review_count[2])

#### Create vocabulary
A vocabulary is created, based on all reviews in the training dataset (the corpus).
Note that the test data is not used to create the vocabulary as this is 'held out' until the end of the modelling process to test how well the model performs on unseen data.

In [ ]:
# Create the vocabulary 
vocab_count = Counter([token for review in train for token in review['tokens']])
print('The vocabulary has {:,} tokens'.format(len(vocab_count)))
print('Some examples are: ')
print('{:6s} {:s} '.format('Count', 'Token'))
for token in random.sample(list(vocab_count), 50):
    print('{1:6,} {0:s} '.format(token, vocab_count[token]))

The vocabulary is large, with many unique tokens. The code below reduces the vocabulary to only include tokens that appear a certain number of times in the training corpus. The count of each word is not needed after this point so this is also dropped in the code below. 

The minimum count for a token to be retained in the vocabulary is a model parameter that can be varied later when tuning the model.

In [ ]:
# Define the minimum count for a token to be retained.
min_count = 15

# Reduce the size of the vocabulary.
vocab = {token for token, count in vocab_count.items() if count >= min_count}

print('There are {:,} tokens used {:n} or more times. Some examples are:'.
      format(len(vocab), min_count))
for token in random.sample(vocab, 20):
    print(token)

In [ ]:
# Convert the reduced vocab into a dictionary to make it easier to check each column
# of the matrix later.
vocab = {token: k for k, token in enumerate(vocab)}

#### Vectorise

This section vectorises the tokens in the corpus using the Bag of Words approach. This involves creating a vector, for each review, that is as long as the corpus's vocabulary. The vector for each review shows how often each word in the vocabulary appears in the review.

For example, assume a vocabulary contains the letters 'a' to 'e'. The text 'a, b, b, c, d' would be vectorised as follows:
{a:1, b:2, c:1, d:1, e:0}.


In [ ]:
# Vectorise the training and test data.
train_X = vectorise_data(train, vocab)
test_X = vectorise_data(test,vocab)

In [ ]:
# Create a scaler so that the features in the dataset (the vectorised tokens)
# can be scaled to have a mean of 0 and a standard deviation of 1.
# Even though the features in this dataset are all on the same scale already,
# this step can sometimes help a model converge on an optimal solution
# faster during the fitting process.
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X)
test_X_scaled = scaler.fit_transform(test_X)

## Modelling

A logistic regression model is now trained. The aim of this model is to predict whether a review is a complaint (i.e. has less than 3 stars) based on the vectorised features for each review.

In [ ]:
# Specify the model.
model = LogisticRegression(C = 5, solver = 'sag', verbose=1, max_iter=1000)
  # This line of code creates a logistic regression model.
  # 'C' is the inverse of regularisation strength and must be a positive float;
  # smaller values of C specify stronger regularisation.
  # solver = 'sag' specifies that the stochastic average gradient descent solver
  # should be used to optimise the parameters of the model.  
  # The model can be tuned by selecting different values for C and
  # also by specifying other parameters for the LogisticRegression module.

# Fit the model.
model.fit(train_X, train_y)
  # The training data can be supplied in several formats including
  # a list of lists (a matrix) for the X data and a list for the y data or
  # numpy arrays.


The warning message given above suggests that the model did not converge on an optimal solution after the maximum number of iterations (100) was performed. You could experiment with the model parameters to see if an optimal solution can be found.
Either way, the evaluation section below suggests that the solution found, while not necessarily optimal, is still reasonably good.

In [ ]:
# Make predictions using the reviews in the test dataset.
pred_y = model.predict(test_X)

## Evaluate model
The next section prints key metrics for the predictions.

In [ ]:
# Use the return_metrics and print_scores functions defined at the top
# of the notebook to review how well this logistic regression model
# has predicted complaints and non-complaints.
scores =  return_metrics(test_y, pred_y)
print_scores(scores, confused = True)

The results are reasonably good, with all printed metrics in the order of 90% or higher.

This suggests that the model has identified a high proportion of both true complaints and true non-complaints and a low proportion of false complaints and false non-complaints.

In [ ]:
# Print a ROC curve and calculate the AUC for the model.
print('The AUC for the model is: ', metrics.roc_auc_score(test_y, pred_y))

# Retrieve the sensitivity (true positive rate) and false positive rate for
# each threshold value (where the threshold value decides how high the
# predicted probability of being a complaint has to be before an observation
# is classified as a complaint.)

false_positive_rate, true_positive_rate, threshold = metrics.roc_curve(test_y, pred_y)
plt.subplots(1, figsize=(5,5))
plt.title('Receiver Operating Characteristic - review classifier')
plt.plot(false_positive_rate, true_positive_rate,color='dodgerblue')
plt.plot([0, 1], ls="--",color='black')
plt.ylabel('Sensitivity (True Positive Rate)')
plt.xlabel('False Positive Rate')
plt.show()

It is instructive to see which tokens have the greatest and smallest weights in the fitted model. The greatest weights imply that these words predict complaints while the smallest weights show that these words predict non-complaints.

In [ ]:
# Retrieve the weights from the model for each token in the vocabulary.
def print_by_weight(model, vocab, k = 5, verbose = True):
    mod_coefs = model.coef_
    
    # Create a list of vocab words and their weights.
    token_weight = list()
    for token, count in vocab.items():
        token_weight.append((token, mod_coefs[0, count]))
   
    # Sort and print the top k items.
    def print_list(tokens):
        print('{:20s}  {:>7s}'.format('Word', 'Weight'))
        print('{:20s}  {:>7s}'.format('----', '------'))
        for token in tokens:
            print ('{:20s}  {:7.1f}'.format(*token))
        print()
        return

    token_weight.sort(key = lambda x : x[1], reverse = True)
    complaint_keywords = token_weight[:k]
    print('Words driving complaints:')
    print_list(token_weight[:k])
    
    token_weight.sort(key = lambda x : x[1])
    non_complaint_keywords = token_weight[:k]
    print('Words driving non-complaints:')
    print_list(token_weight[:k])

    return complaint_keywords, non_complaint_keywords

complaint_keywords, non_complaint_keywords = print_by_weight(model, vocab, 10)

## Comments

The model fitted provides a reasonable prediction of whether each review is a complaint (less than 3 stars) or a non-complaint (3 or more stars). The output of the model also makes sense in that the top 'complaint' words all have clear negative sentiment and the top 'non-complaint' words all have clear positive sentiment.

As an exercise, you should experiment with the tokenisation of reviews by trying the following:
- remove standard stop words; and
- perform stemming or lemmatisation.

You may like to review the code in Case Study 2 first and then come back to try the above.